In [6]:
import boto3
import numpy as np
from decimal import Decimal

# Initialize DynamoDB
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('documents-vec-db')

# Example embedding vector (1024-dim)
embedding_vector = np.random.rand(1024)

# Convert each float to Decimal
embedding_vector_decimal = [Decimal(str(x)) for x in embedding_vector]

# Insert test document
table.put_item(
    Item={
        "doc_id": '2',
        "content": "AWS Bedrock enables generative AI using foundation models.",
        "embedding": embedding_vector_decimal,
        "source": "aws-bedrock-docs",
        "doc_type": "text"
    }
)

print("Test document inserted into DynamoDB (Decimal conversion done).")


Test document inserted into DynamoDB (Decimal conversion done).


In [9]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

# AWS session
session = boto3.Session()
credentials = session.get_credentials()
region = session.region_name

awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    "aoss",
    session_token=credentials.token
)

# OpenSearch Serverless connection
client = OpenSearch(
    hosts=[{"host": "oyw6m139z8kur98k8l96.us-east-1.aoss.amazonaws.com", "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)


print("Connected to OpenSearch Serverless!")


Connected to OpenSearch Serverless!


In [10]:
index_name = "rag_chunks_index"

# Define mapping for vector + metadata
index_body = {
    "settings": {},
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "content": {"type": "text"},
            "embedding": {"type": "knn_vector", "dimension": 1024, "method": {"name": "hnsw", "space_type": "l2", "engine": "nmslib"}},
            "source": {"type": "keyword"},
            "doc_type": {"type": "keyword"}
        }
    }
}

# Create index if not exists
if not client.indices.exists(index=index_name):
    client.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created!")
else:
    print(f"Index '{index_name}' already exists.")


AuthorizationException: AuthorizationException(403, '')

In [11]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

# AWS session and credentials
session = boto3.Session()
credentials = session.get_credentials()
region = session.region_name

awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    "aoss",
    session_token=credentials.token
)

# OpenSearch Serverless endpoint (no https://, no brackets)
client = OpenSearch(
    hosts=[{"host": "oyw6m139z8kur98k8l96.us-east-1.aoss.amazonaws.com", "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Test connection
try:
    info = client.info()
    print("✅ Connection successful!")
    print(info)
except Exception as e:
    print("❌ Connection failed:", e)


❌ Connection failed: NotFoundError(404, '')


In [12]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

session = boto3.Session()
credentials = session.get_credentials()
region = session.region_name

awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region,
    "aoss",
    session_token=credentials.token
)

client = OpenSearch(
    hosts=[{"host": "oyw6m139z8kur98k8l96.us-east-1.aoss.amazonaws.com", "port": 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# List all indexes in the collection
try:
    indices = client.indices.get_alias("*")
    print("✅ Connection works! Indexes found:", list(indices.keys()))
except Exception as e:
    print("❌ Connection test failed:", e)


❌ Connection test failed: IndicesClient.get_alias() takes 1 positional argument but 2 positional arguments (and 2 keyword-only arguments) were given


In [34]:
import boto3
import json
import os
import sys

import boto3
import botocore
import pandas as pd

from opensearchpy import OpenSearch


##opensearch configs
host = 'search-rag-kg-store-kagetbbzgjjrkpdh444lid4oyi.us-east-1.es.amazonaws.com'  
port = 443 
auth = ('admin', 'Test$1234')
index_name = "store_documents_src"


##creating opensearch client
op_client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True
)


# ##reading titles using pandas
# df = pd.read_csv('netflix_titles.csv')

# selected_columns = df[['show_id','title','description']]

# refined_df = selected_columns.head(100)




import boto3
import json
import numpy as np
from numpy.linalg import norm


client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)


# def generate_embedding(input_text):
    
#     # Set the model ID, e.g., Titan Text Embeddings V2.
#     model_id = "amazon.titan-embed-text-v2:0"

#     # Create the request for the model.
#     native_request = {"inputText": input_text}

#     # Convert the native request to JSON.
#     request = json.dumps(native_request)

#     # Invoke the model with the request.
#     response = client.invoke_model(modelId=model_id, body=request)

#     # Decode the model's native response body.
#     model_response = json.loads(response["body"].read())

#     # Extract and print the generated embedding and the input text token count.
#     embedding = np.array(model_response["embedding"])
#     # embedding = model_response["embedding"]

#     input_token_count = model_response["inputTextTokenCount"]

#     return embedding

# def cosine_similarity(vec1, vec2):
#     return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))


##generate embedding using titan model
def generate_embedding(value):
    try:
        body = json.dumps({"inputText": value})
        modelId = "amazon.titan-embed-text-v1"
        accept = "application/json"
        contentType = "application/json"

        response = client.invoke_model(
            body=body, modelId=modelId, accept=accept, contentType=contentType
        )
        response_body = json.loads(response.get("body").read())

        return response_body['embedding']
    except botocore.exceptions.ClientError as error:
        print(error)



documents = [
    "AWS Bedrock provides foundation models for GenAI",
    "Amazon Bedrock is used to build AI-powered applications",
    "S3 is an object storage service",
    "EC2 provides scalable virtual servers",
    "I love playing football on weekends"
]



doc_embeddings = [generate_embedding(doc) for doc in documents]




# ##creating a document to insert
# def create_document(show_id,title,description):
#     document = {
#         'show_id':show_id,
#         'title':title['embedding'],
#         'description':description['embedding']
#     }

#     insert_document(document)

# ##inserting document into opensearch
# def insert_document(document):
#     client.index(index=index_name, body=document)


# ##iterating thorough each row in data frame created through pandas and requesting embedding
# for index, row in refined_df.iterrows():
#     show_id = row['show_id']
#     title = row['title']
#     description = row['description']
#     title_embedding = generate_embedding(title)
#     description_embedding = generate_embedding(description)
#     create_document(show_id,title_embedding,description_embedding)
#     print(f"inserted:{index}")


doc_id = 0 
for doc, emb in zip(documents,doc_embeddings):

    doc_id+=1

    document_write = {
        'document_id':str(doc_id),
        'document_text':doc,
        'embeddings':emb
    }

    op_client.index(index=index_name, body=document_write)




In [35]:
len(doc_embeddings[0])

1536

In [ ]:
def run_query(query_embedding):
    query = {
        "size": 2,
         "_source": "document_text", 
        "query": {
            "knn": {
                "embeddings": {
                    "vector": query_embedding,
                    "k": 2
            }
        }
    }   }
    response = op_client.search(index=index_name, body=query)


    print(response)

In [26]:
query =  "How can I build generative AI apps on AWS?"

query_embedding = generate_embedding(query)

In [28]:
len(query_embedding)

1024

In [27]:
run_query(query_embedding)

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPSConnectionPool(host='search-rag-kg-store-kagetbbzgjjrkpdh444lid4oyi.us-east-1.es.amazonaws.com', port=443): Read timed out. (read timeout=10))

In [29]:
doc_embeddings[0]

[-0.07949460297822952,
 0.030731264501810074,
 -0.02727186307311058,
 -0.02470552548766136,
 -0.045918285846710205,
 0.03651558235287666,
 -0.0007422379567287862,
 -0.06103047356009483,
 -0.004920692648738623,
 -0.03857846558094025,
 -0.014252245426177979,
 0.022503286600112915,
 -0.02800152823328972,
 0.004751893226057291,
 -0.03129654750227928,
 -0.023151861503720284,
 0.011659901589155197,
 -0.03913196921348572,
 0.051816198974847794,
 0.06175302341580391,
 0.014440671540796757,
 0.054469142109155655,
 0.017102694138884544,
 0.011674132198095322,
 0.006063031032681465,
 -0.02420109324157238,
 -0.036562688648700714,
 -0.043070290237665176,
 -0.0053103044629096985,
 -0.006639107596129179,
 0.008288824930787086,
 0.03486979007720947,
 -0.049595557153224945,
 0.01225364301353693,
 0.05425912141799927,
 -0.06444203108549118,
 0.14650984108448029,
 -0.02944527566432953,
 -0.06131630390882492,
 0.03967762365937233,
 0.021629849448800087,
 0.054367076605558395,
 -0.0013997697969898582,
 0.0